## Cell 1 — Check Device

In [ ]:
import torch
import platform
from pathlib import Path

print(f'Python  : {platform.python_version()}')
print(f'PyTorch : {torch.__version__}')
print(f'Machine : {platform.machine()} — {platform.system()}')
print()

if torch.backends.mps.is_available():
    DEVICE = 'mps'
    print('✅ Apple MPS (M2 GPU) — training will use your M2 chip')
elif torch.cuda.is_available():
    DEVICE = 'cuda'
    print('✅ CUDA GPU available')
else:
    DEVICE = 'cpu'
    print('⚠️  No GPU found — using CPU')

print(f'\n🖥️  Device: {DEVICE}')

## Cell 2 — Settings

In [ ]:
from pathlib import Path

# ── Paths ──────────────────────────────────────────────────────
BASE_DIR     = Path.cwd()                   
DATASET_YAML = BASE_DIR / 'dataset.yaml'     
SPLITS_DIR   = BASE_DIR / 'data' / 'splits'
RUNS_DIR     = BASE_DIR / 'runs'
REPORTS_DIR  = BASE_DIR / 'reports'

MODEL_SIZE  = 'yolov8n.pt'
MODEL_NAME  = 'spinach_v1'


# ── Training — FAST settings for M2 Air ────────────────────────
EPOCHS      = 20      # 20 is enough for a first run with pretrained weights
BATCH_SIZE  = 32      # 32 is fine on 16GB M2 — halves training time vs batch=8
IMAGE_SIZE  = 416     # smaller than 640, still good quality, faster
PATIENCE    = 7       # stop early if no improvement for 7 epochs
WORKERS     = 0       
AMP         = False   # MUST be False on MPS 

# ── Classes ────────────────────────────────────────────────────
CLASS_NAMES = ['spinach_leaf','stem','GOOD','YELLOW','HOLE','TRACK','WSPOT','FSPOT']
CLASS_COLORS = [
    (46,125,50),   # spinach_leaf
    (139,90,43),   # stem
    (76,175,80),   # GOOD
    (255,193,7),   # YELLOW
    (244,67,54),   # HOLE
    (255,87,34),   # TRACK
    (156,39,176),  # WSPOT
    (63,81,181),   # FSPOT
]

# ── Verify ─────────────────────────────────────────────────────
if DATASET_YAML.exists():
    print(f'dataset.yaml found')
    print(f'Config ready')
    print(f'   Model      : {MODEL_SIZE} (nano — fast mode)')
    print(f'   Device     : {DEVICE}')
    print(f'   Epochs     : {EPOCHS}')
    print(f'   Batch      : {BATCH_SIZE}')
    print(f'   Image size : {IMAGE_SIZE}px')
    print(f'   Est. time  : 20–35 minutes')
else:
    print(f'Dataset.yaml not found at {DATASET_YAML}')
    print('   Run: python 01_prepare_dataset.py first')

## Cell 3 — Check the Dataset

In [ ]:
from collections import Counter

print('📊 Dataset Check')
print('─' * 45)
all_ok = True
for split in ['train', 'val', 'test']:
    imgs = len(list((SPLITS_DIR / split / 'images').glob('*.*')))
    lbls = len(list((SPLITS_DIR / split / 'labels').glob('*.txt')))
    ok   = '✅' if imgs > 0 else '❌'
    print(f'  {ok} {split:<6} : {imgs} images | {lbls} labels')
    if imgs == 0:
        all_ok = False

if all_ok:
    # Class breakdown for training set
    counts = Counter()
    for f in (SPLITS_DIR / 'train' / 'labels').glob('*.txt'):
        for line in f.read_text().splitlines():
            parts = line.strip().split()
            if parts:
                counts[int(parts[0])] += 1
    print(f'\n  Training set class breakdown:')
    for i, name in enumerate(CLASS_NAMES):
        n = counts.get(i, 0)
        print(f'    [{i}] {name:<15} {n:>5} annotations')
    print('\n  ✅ Ready to train!')
else:
    print('\n  ❌ Missing data. Run 01_prepare_dataset.py first.')

## Cell 4 — Preview Training Images



In [ ]:
import cv2
import matplotlib.pyplot as plt
import random

def draw_boxes(img_path, lbl_path):
    img = cv2.imread(str(img_path))
    if img is None:
        return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    if lbl_path.exists():
        for line in lbl_path.read_text().splitlines():
            parts = line.strip().split()
            if len(parts) == 5:
                cls = int(parts[0])
                cx, cy, bw, bh = float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])
                x1 = int((cx - bw/2) * w)
                y1 = int((cy - bh/2) * h)
                x2 = int((cx + bw/2) * w)
                y2 = int((cy + bh/2) * h)
                col = CLASS_COLORS[cls % len(CLASS_COLORS)]
                cv2.rectangle(img, (x1,y1), (x2,y2), col, 3)
                label = CLASS_NAMES[cls] if cls < len(CLASS_NAMES) else str(cls)
                (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                cv2.rectangle(img, (x1, max(0,y1-22)), (x1+tw+4, y1), col, -1)
                cv2.putText(img, label, (x1+2, max(15, y1-5)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
    return img

train_imgs = list((SPLITS_DIR / 'train' / 'images').glob('*.*'))

if not train_imgs:
    print('No training images found — check that 01_prepare_dataset.py ran successfully.')
else:
    sample = random.sample(train_imgs, min(9, len(train_imgs)))
    fig, axes = plt.subplots(3, 3, figsize=(15, 13))

    for ax, img_path in zip(axes.flatten(), sample):
        lbl_path = SPLITS_DIR / 'train' / 'labels' / (img_path.stem + '.txt')
        img = draw_boxes(img_path, lbl_path)
        if img is not None:
            ax.imshow(img)
            cls_ids = []
            if lbl_path.exists():
                for line in lbl_path.read_text().splitlines():
                    parts = line.strip().split()
                    if parts:
                        cls_ids.append(CLASS_NAMES[int(parts[0])])
            ax.set_title(', '.join(set(cls_ids)) if cls_ids else 'no label', fontsize=9, fontweight='bold')
        ax.axis('off')

    plt.suptitle('Sample Training Images', fontsize=13, fontweight='bold')
    plt.tight_layout()
    plt.savefig(REPORTS_DIR / 'sample_training_images.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f'Saved {len(sample)} sample images to reports/sample_training_images.png')
    print('Boxes should be visible on each image with class names shown above them.')
    print('If the box covers the whole image — that is normal for this dataset format.')

## Cell 5 — Train

This will take around 20–35 minutes on an M2 MacBook Air.
Plug in your charger and leave it running — the best weights are saved automatically to `runs/spinach_v1/weights/best.pt`.
If you need to rerun this cell, just run it again — it overwrites the previous output automatically.
To keep a previous run alongside a new one, change `MODEL_NAME` to `spinach_v2` in Cell 2 first.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import random
from pathlib import Path

def draw_boxes(img_path, lbl_path):
    img = cv2.imread(str(img_path))
    if img is None:
        return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    if lbl_path.exists():
        for line in lbl_path.read_text().splitlines():
            parts = line.strip().split()
            if len(parts) == 5:
                cls = int(parts[0])
                cx, cy, bw, bh = float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])
                x1 = int((cx - bw/2) * w)
                y1 = int((cy - bh/2) * h)
                x2 = int((cx + bw/2) * w)
                y2 = int((cy + bh/2) * h)
                col = CLASS_COLORS[cls % len(CLASS_COLORS)]
                cv2.rectangle(img, (x1,y1), (x2,y2), col, 3)
                label = CLASS_NAMES[cls] if cls < len(CLASS_NAMES) else str(cls)
                # Background rectangle for text readability
                (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                cv2.rectangle(img, (x1, max(0,y1-22)), (x1+tw+4, y1), col, -1)
                cv2.putText(img, label, (x1+2, max(15, y1-5)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
    return img

train_imgs = list((SPLITS_DIR / 'train' / 'images').glob('*.*'))

if not train_imgs:
    print('No training images found. Run 01_prepare_dataset.py first.')
else:
    sample = random.sample(train_imgs, min(9, len(train_imgs)))
    fig, axes = plt.subplots(3, 3, figsize=(15, 13))

    for ax, img_path in zip(axes.flatten(), sample):
        lbl_path = SPLITS_DIR / 'train' / 'labels' / (img_path.stem + '.txt')
        img = draw_boxes(img_path, lbl_path)
        if img is not None:
            ax.imshow(img)
            # Show class name from label file
            cls_ids = []
            if lbl_path.exists():
                for line in lbl_path.read_text().splitlines():
                    parts = line.strip().split()
                    if parts:
                        cls_ids.append(CLASS_NAMES[int(parts[0])])
            title = ', '.join(set(cls_ids)) if cls_ids else 'no label'
            ax.set_title(title, fontsize=9, fontweight='bold')
        ax.axis('off')

    plt.suptitle('Sample Training Images — Verify Before Training', fontsize=13, fontweight='bold')
    plt.tight_layout()
    plt.savefig(REPORTS_DIR / 'sample_training_images.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f'Showing {len(sample)} random training images')
    print(f'   Saved to: reports/sample_training_images.png')
    print()
    print('What to check:')
    print('  Images show spinach/leaves')
    print('  Coloured boxes are drawn on the images')
    print('  Class labels match what you see in the image')
    print('  If boxes cover the entire image — that is expected (classification→detection conversion)')
    print('  If no boxes appear — check your labels folder')

## Cell 6 — Evaluate on Test Set

Runs the trained model against images it has never seen before (the held-out test set).
This gives a fair picture of how well it actually generalises.
Any class showing a warning here is worth looking into — see Cell 9 for options.

In [ ]:
from ultralytics import YOLO

print('🚀 Starting training...')
print(f'   Model      : {MODEL_SIZE}')
print(f'   Device     : {DEVICE}')
print(f'   Epochs     : {EPOCHS}')
print(f'   Batch      : {BATCH_SIZE}')
print(f'   Image size : {IMAGE_SIZE}')
print(f'   Early stop : after {PATIENCE} epochs no improvement')
print()

model = YOLO(MODEL_SIZE)   # loads pretrained nano weights from COCO

results = model.train(
    # ── Dataset ──────────────────────────────────────────
    data          = str(DATASET_YAML),
    # ── Hardware ─────────────────────────────────────────
    device        = DEVICE,
    workers       = WORKERS,       
    amp           = AMP,           
    # ── Training ─────────────────────────────────────────
    epochs        = EPOCHS,
    batch         = BATCH_SIZE,
    imgsz         = IMAGE_SIZE,
    patience      = PATIENCE,
    # ── Learning rate ────────────────────────────────────
    lr0           = 0.01,
    lrf           = 0.001,
    warmup_epochs = 3,
    # ── Built-in augmentation ────────────────────────────
    mosaic        = 1.0,
    degrees       = 15.0,
    fliplr        = 0.5,
    flipud        = 0.1,
    hsv_h         = 0.015,
    hsv_s         = 0.7,
    hsv_v         = 0.4,
    # ── Output ───────────────────────────────────────────
    project       = str(RUNS_DIR),
    name          = MODEL_NAME,
    save          = True,
    plots         = True,
    exist_ok      = True,          # overwrites previous run — no need to delete
)

BEST_MODEL = RUNS_DIR / MODEL_NAME / 'weights' / 'best.pt'
print()
print('=' * 50)
print('Training complete!')
print(f'   Best model saved : {BEST_MODEL}')
print('   Next: run Cell 5 to evaluate')
print('=' * 50)

## Cell 7 — Training Curves 

Plots the loss and mAP over each epoch. Useful for checking if the model trained properly.
Loss should decrease over time and mAP should increase — if they look erratic something went wrong.
The chart is also saved to `reports/training_curves.png`.

In [ ]:
from ultralytics import YOLO

BEST_MODEL = RUNS_DIR / MODEL_NAME / 'weights' / 'best.pt'

if not BEST_MODEL.exists():
    print(f'Model not found. Run Cell 4 first.')
else:
    print(f'Loading: {BEST_MODEL}')
    eval_model = YOLO(str(BEST_MODEL))

    print('\n🔍 Evaluating on test set...')
    metrics = eval_model.val(
        data     = str(DATASET_YAML),
        split    = 'test',
        imgsz    = IMAGE_SIZE,
        conf     = 0.25,
        iou      = 0.5,
        device   = DEVICE,
        workers  = 0,
        plots    = True,
        project  = str(RUNS_DIR),
        name     = MODEL_NAME + '_eval',
        exist_ok = True,
    )

    print()
    print('=' * 50)
    print('TEST SET RESULTS')
    print('=' * 50)
    print(f'  mAP@0.5      : {metrics.box.map50:.4f}   (target ≥ 0.60 for nano)')
    print(f'  mAP@0.5:0.95 : {metrics.box.map:.4f}   (target ≥ 0.40)')
    print(f'  Precision    : {metrics.box.mp:.4f}   (target ≥ 0.70)')
    print(f'  Recall       : {metrics.box.mr:.4f}   (target ≥ 0.65)')
    print('=' * 50)

    print('\nPer-class AP@0.5:')
    print('─' * 45)
    weak = []
    for i, (name, ap) in enumerate(zip(CLASS_NAMES, metrics.box.ap50)):
        flag = '✅' if ap >= 0.50 else '⚠️ '
        print(f'  {flag} [{i}] {name:<15} {ap:.4f}')
        if ap < 0.50:
            weak.append(name)

    print()
    if weak:
        print(f'  ⚠️  Weak classes (AP < 0.50): {", ".join(weak)}')
        print('     → Run Cell 8 (improve) or upgrade to yolov8s in Cell 2')
    else:
        print('   All classes above 0.50 — good result for nano model!')

## Cell 8 — Confusion Matrix 

Shows which classes the model is getting confused between.
The diagonal should be as dark as possible — that means correct predictions.
If FSPOT and WSPOT are confused with each other for example, you would see it clearly here.
Run Cell 6 first — this reads the output from that evaluation.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

csv_path = RUNS_DIR / MODEL_NAME / 'results.csv'

if not csv_path.exists():
    print('Run Cell 4 first.')
else:
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip()

    fig, axes = plt.subplots(2, 3, figsize=(16, 9))
    plots = [
        ('train/box_loss',        'Train Box Loss',    '#e53935'),
        ('train/cls_loss',        'Train Class Loss',  '#fb8c00'),
        ('val/box_loss',          'Val Box Loss',      '#1e88e5'),
        ('val/cls_loss',          'Val Class Loss',    '#00acc1'),
        ('metrics/mAP50(B)',      'mAP@0.5',           '#43a047'),
        ('metrics/precision(B)',  'Precision',         '#8e24aa'),
    ]
    for ax, (col, title, colour) in zip(axes.flatten(), plots):
        c = col if col in df.columns else col.replace('(B)','')
        if c in df.columns:
            ax.plot(df['epoch'], df[c], color=colour, linewidth=2)
            ax.set_title(title, fontweight='bold')
            ax.set_xlabel('Epoch')
            ax.grid(True, alpha=0.3, linestyle='--')
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
        else:
            ax.set_title(f'{title} (not found)', color='grey')
            ax.axis('off')

    plt.suptitle('Training Progress — Spinach Defect Detection', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(REPORTS_DIR / 'training_curves.png', dpi=150)
    plt.show()
    print('Saved: reports/training_curves.png')

## Cell 10 — Upgrade to a Better Model 

If the results from Cell 6 are not good enough, this switches to `yolov8s` (small) which is more accurate.
It takes around 60–90 minutes but the improvement is usually noticeable.
Change `RUN_UPGRADE` to `True` below and run the cell.

In [ ]:
# import cv2
# import matplotlib.pyplot as plt
# import random
# from ultralytics import YOLO

# BEST_MODEL = RUNS_DIR / MODEL_NAME / 'weights' / 'best.pt'

# MY_IMAGE       = 'BASE_DIR/data/spinach.jpg'
# USE_TEST_IMAGE = True   # uses a random image from your test set
# # ────────────────────────────────────────────────────────────────────────

# if not BEST_MODEL.exists():
#     print('Run Cell 4 first.')
# else:
#     infer_model = YOLO(str(BEST_MODEL))
#     if USE_TEST_IMAGE:
#         test_imgs = list((SPLITS_DIR / 'test' / 'images').glob('*.*'))
#         source = str(random.choice(test_imgs))
#         print(f'Using: {source}')
#     else:
#         source = MY_IMAGE

#     results = infer_model.predict(
#         source  = source,
#         conf    = 0.30,
#         imgsz   = IMAGE_SIZE,
#         device  = DEVICE,
#         verbose = False,
#     )

#     for result in results:
#         ann = cv2.cvtColor(result.plot(), cv2.COLOR_BGR2RGB)
#         orig = cv2.cvtColor(cv2.imread(result.path), cv2.COLOR_BGR2RGB)

#         fig, axes = plt.subplots(1, 2, figsize=(14, 6))
#         axes[0].imshow(orig);  axes[0].set_title('Original');    axes[0].axis('off')
#         axes[1].imshow(ann);   axes[1].set_title(f'Detections ({len(result.boxes)})'); axes[1].axis('off')
#         plt.suptitle('Spinach Defect Detection', fontsize=13, fontweight='bold')
#         plt.tight_layout()
#         plt.show()

#         print(f'\nDetected {len(result.boxes)} object(s):')   
#         for box in result.boxes:
#             cls_id = int(box.cls[0])
#             print(f'  [{cls_id}] {CLASS_NAMES[cls_id]:<15} confidence: {float(box.conf[0]):.1%}')

## Cell 11 — Export Model 

In [ ]:
from ultralytics import YOLO

BEST_MODEL = RUNS_DIR / MODEL_NAME / 'weights' / 'best.pt'

if not BEST_MODEL.exists():
    print('❌ Run Cell 4 first.')
else:
    m = YOLO(str(BEST_MODEL))
    path = m.export(format='onnx', imgsz=IMAGE_SIZE)
    print(f'✅ Exported: {path}')
    print('\nTo use this model anywhere:')
    print(f'  model = YOLO("{BEST_MODEL}")')
    print('  results = model.predict("spinach.jpg", conf=0.3)')